In [1]:
import pprint as pp

from src.options import Options
from src.utils import setup
from main import run as main
from src.utils.hyperparemer_tuning_config import hyperparameter_config

import ray
from ray import tune, air
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.air import session

In [3]:
def run(hyperparameter_config: dict):
    # Pretty print the run args
    hyperparameter_config["data_dir"] = '/home/kfragkedaki/projects/Pedestrian_Project/resources/SinD/Data'
    hyperparameter_config["data_class"] = 'sind'
    hyperparameter_config["pattern"] = 'Ped_smoothed_tracks'
    hyperparameter_config["data_class"] = 'sind'
    hyperparameter_config["pos_encoding"] = 'learnable'
    hyperparameter_config["name"] = 'SINDDataset_pretrained'
    hyperparameter_config["comment"] = 'pretraining_through_imputation-hyperparameter_tuning'
    hyperparameter_config["output_dir"] = '/home/kfragkedaki/projects/Pedestrian_Project/ray_results'
    hyperparameter_config["batch_size"] = 512
    hyperparameter_config["epochs"] = 2

    args_list = [f"--{k}={v}" for k, v in hyperparameter_config.items()]
    args_list.append("--no_cuda")
    args_list.append("--hyperparameter_tuning")
    args_list.append("--harden")
    
    opts = Options().parse(args_list) 

    pp.pprint(vars(opts))

    main(setup(opts) , session)

In [4]:
N_ITER = 10
ray.init(num_cpus=14)
searcher = HyperOptSearch(
    space=hyperparameter_config, metric="loss", mode="min", n_initial_points=int(N_ITER / 10)
)
algo = ConcurrencyLimiter(searcher, max_concurrent=3)
objective = tune.with_resources(
    tune.with_parameters(run), resources={"cpu": 14}
)

tuner = tune.Tuner(
    trainable=objective,
    run_config=air.RunConfig(),
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        search_alg=algo,
        num_samples=N_ITER,
    ),
)

results = tuner.fit()
ray.shutdown()

(run pid=408936) {'activation': 'relu',
(run pid=408936)  'batch_size': 512,
(run pid=408936)  'comment': 'pretraining_through_imputation-hyperparameter_tuning',
(run pid=408936)  'config_filepath': None,
(run pid=408936)  'console': False,
(run pid=408936)  'data_chunk_len': 140,
(run pid=408936)  'data_class': 'sind',
(run pid=408936)  'data_dir': '/home/kfragkedaki/projects/Pedestrian_Project/resources/SinD/Data',
(run pid=408936)  'data_normalization': 'none',
(run pid=408936)  'dropout': 0.15000000000000002,
(run pid=408936)  'early_stopping_delta': 0.0,
(run pid=408936)  'early_stopping_patience': None,
(run pid=408936)  'embedding_dim': 128,
(run pid=408936)  'epochs': 2,
(run pid=408936)  'eval_only': False,
(run pid=408936)  'exclude_feats': None,
(run pid=408936)  'experiment_name': 'SINDDataset_pretrained',
(run pid=408936)  'harden': True,
(run pid=408936)  'harden_step': 15,
(run pid=408936)  'hidden_dim': 512,
(run pid=408936)  'hyperparameter_tuning': True,
(run pid=4089

(run pid=408936) 2024-03-13 22:25:02,453 | INFO : Stored configuration file in '/home/kfragkedaki/projects/Pedestrian_Project/ray_results/SINDDataset_pretrained_2024-03-13_22-25-02_BDQ'
(run pid=408936) 2024-03-13 22:25:02,454 | INFO : Running:
(run pid=408936) /home/kfragkedaki/miniconda3/envs/pedestrian_project/lib/python3.9/site-packages/ray/_private/workers/default_worker.py --node-ip-address=172.30.124.111 --node-manager-port=42715 --object-store-name=/tmp/ray/session_2024-03-13_22-24-57_355499_407966/sockets/plasma_store --raylet-name=/tmp/ray/session_2024-03-13_22-24-57_355499_407966/sockets/raylet --redis-address=None --temp-dir=/tmp/ray --metrics-agent-port=65293 --runtime-env-agent-port=50319 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --runtime-env-agent-port=50319 --gcs-address=172.30.124.111:63144 --session-name=session_2024-03-13_22-24-57_355499_407966 --temp-dir=/tmp/ray --webui= --cluster-id=1a424adfba121161a6e6cb85bf7fae0f41acae91f969e4e8a326f2df -

(run pid=408936) max_seq_len 350
(run pid=408936) final max_seq_len 140


(run pid=408936) 2024-03-13 22:25:02,826 | INFO : 1021 samples may be used for training
(run pid=408936) 2024-03-13 22:25:02,826 | INFO : 256 samples will be used for validation
(run pid=408936) /home/kfragkedaki/miniconda3/envs/pedestrian_project/lib/python3.9/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
(run pid=408936)   warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
(run pid=408936) 2024-03-13 22:25:02,852 | INFO : Model:
(run pid=408936) TSTransformerEncoder(
(run pid=408936)   (project_inp): Linear(in_features=6, out_features=128, bias=True)
(run pid=408936)   (pos_enc): LearnablePositionalEncoding(
(run pid=408936)     (dropout): Dropout(p=0.15000000000000002, inplace=False)
(run pid=408936)   )
(run pid=408936)   (transformer_encoder): TransformerEncoder(
(run pid=4089

(run pid=408936) Evaluating Epoch 0   0.0% | batch:         0 of         1	|	loss: 1616.97


...

(run pid=408936) 2024-03-13 22:26:56,394 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.36751699447631836 seconds
(run pid=408936) 
(run pid=408936) 2024-03-13 22:26:56,394 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.4103950796456173 seconds
(run pid=408936) 2024-03-13 22:26:56,394 | INFO : Avg batch val. time: 0.4103950796456173 seconds
(run pid=408936) 2024-03-13 22:26:56,394 | INFO : Avg sample val. time: 0.0013238550956310236 seconds
(run pid=408936) 2024-03-13 22:26:56,394 | INFO : Epoch 1 Validation Summary: epoch: 1.000000 | loss: 3.116155 | 
(run pid=408936) 2024-03-13 22:26:56,431 | INFO : Learning rate updated to: 0.0007106716060801806
Training Epoch:  50%|█████     | 1/2 [00:04<00:04,  4.69s/it]


(run pid=408936) Training Epoch 2   0.0% | batch:         0 of         3	|	loss: 0.284646
(run pid=408936) Training Epoch 2  33.3% | batch:         1 of         3	|	loss: 0.284673
(run pid=408936) Training Epoch 2  66.7% | batch:         2 of         3	|	loss: 0.294399


(run pid=408936) 2024-03-13 22:27:00,724 | INFO : Epoch 2 Training Summary: epoch: 2.000000 | loss: 0.286305 | 
(run pid=408936) 2024-03-13 22:27:00,724 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 4.292231321334839 seconds
(run pid=408936) 
(run pid=408936) 2024-03-13 22:27:00,725 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 4.289551615715027 seconds
(run pid=408936) 2024-03-13 22:27:00,725 | INFO : Avg batch train. time: 1.4298505385716755 seconds
(run pid=408936) 2024-03-13 22:27:00,725 | INFO : Evaluating on validation set ...


(run pid=408936) Evaluating Epoch 2   0.0% | batch:         0 of         1	|	loss: 1.23035


(run pid=408936) 2024-03-13 22:27:01,015 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.29048728942871094 seconds
(run pid=408936) 
(run pid=408936) 2024-03-13 22:27:01,015 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.4063981533050537 seconds
(run pid=408936) 2024-03-13 22:27:01,015 | INFO : Avg batch val. time: 0.4063981533050537 seconds
(run pid=408936) 2024-03-13 22:27:01,015 | INFO : Avg sample val. time: 0.001310961784855012 seconds
(run pid=408936) 2024-03-13 22:27:01,016 | INFO : Epoch 2 Validation Summary: epoch: 2.000000 | loss: 1.230351 | 
2024-03-13 22:27:01,082	INFO tune.py:1042 -- Total run time: 120.60 seconds (120.57 seconds for the tuning loop).
